In [1]:
# first neural network with keras tutorial
import pandas as pd
import pandas_ta as ta
import random
from collections import deque
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, Activation, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint, Callback, TensorBoard, LearningRateScheduler
import time
import statistics
from sklearn import preprocessing
from matplotlib import pyplot as plt
import os.path
from numpy import loadtxt
import yfinance as yf
import math
pd.options.mode.chained_assignment = None  # default='warn'

In [48]:
## getting data ##
def getDF():
    # Load data
    df = pd.read_csv('BTC-USD_1hour.csv')
    df = df.drop(columns=['Unnamed: 0'])
    df = df.tail(1000)
    
    # add data points #
    
    # closing percents
    df['close_per1'] = df.ta.percent_return(length=1, close = df['close'])*100
    df['close_per3'] = df.ta.percent_return(length=3, close = df['close'])*100
    
    # ema
    df['ema10'] = df.ta.ema(length=10, close = df['close'])
    df['ema3'] = df.ta.ema(length=3, close = df['close'])
    
    # rsi
    df['rsi7'] = df.ta.rsi(length=7, close = df['close'])/100
    df['rsi14'] = df.ta.rsi(length=14, close = df['close'])/100
    
    # willr
    df['williams7'] = df.ta.willr(length=7, close = df['close'])/100
    df['williams14'] = df.ta.willr(length=14, close = df['close'])/100
    
    # stochrsi
    df_stochrsi7 = df.ta.stochrsi(length=7, rsi_length=7)
    df_stochrsi14 = df.ta.stochrsi(length=14, rsi_length=14)
    df_stochrsi7['stochrsi_hist7'] = df_stochrsi7['STOCHRSIk_7_7_3_3'] - df_stochrsi7['STOCHRSId_7_7_3_3']
    df_stochrsi14['stochrsi_hist14'] = df_stochrsi14['STOCHRSIk_14_14_3_3'] - df_stochrsi14['STOCHRSId_14_14_3_3']
    df['stochrsi_hist7'] = (df_stochrsi7['STOCHRSIk_7_7_3_3'] - df_stochrsi7['STOCHRSId_7_7_3_3'])/100
    df['stochrsi_hist14'] = (df_stochrsi14['STOCHRSIk_14_14_3_3'] - df_stochrsi14['STOCHRSId_14_14_3_3'])/100
    
    # BBands %
    df_bbands = df.ta.bbands()
    df['BB_perL'] = -1*((df_bbands['BBM_5_2.0']-df_bbands['BBL_5_2.0'])/df_bbands['BBM_5_2.0'])
    df['BB_perH'] = -1*((df_bbands['BBM_5_2.0']-df_bbands['BBU_5_2.0'])/df_bbands['BBM_5_2.0'])
    

#     del df['timestamp']
    del df['trades']
    del df['volume']
    
    df = df.dropna()
    df.reset_index(inplace=True)
    
    return df

In [49]:
df = getDF()
df.tail(5)

,index,timestamp,open,high,low,close,close_per1,close_per3,ema10,ema3,rsi7,rsi14,williams7,williams14,stochrsi_hist7,stochrsi_hist14,BB_perL,BB_perH
864,1996,"03/05/2022, 18:00:00",39466.30,39468.30,39337.11,39407.02,-0.150204,-0.211671,39369.164537,39423.083528,0.551462,0.451139,-0.586428,-0.258609,-0.253311,-0.027914,-0.002387,0.002387
865,1997,"03/05/2022, 19:00:00",39407.02,39611.31,39361.85,39602.27,0.495470,0.584474,39411.547349,39512.676764,0.678469,0.518709,-0.027262,-0.013537,0.070809,0.031619,-0.004493,0.004493
866,1998,"03/05/2022, 20:00:00",39602.27,39686.74,39441.29,39458.78,-0.362328,-0.019054,39420.135103,39485.728382,0.545930,0.472657,-0.576006,-0.306719,-0.025230,-0.024314,-0.004447,0.004447
867,1999,"03/05/2022, 21:00:00",39458.78,39533.00,39383.49,39529.36,0.178870,0.310452,39439.994176,39507.544191,0.591702,0.496343,-0.397665,-0.211754,0.009664,-0.019931,-0.003801,0.003801
868,2000,"03/05/2022, 22:00:00",39529.36,39569.60,39284.67,39373.74,-0.393682,-0.577063,39427.947962,39440.642096,0.469865,0.448508,-0.778471,-0.432016,-0.188798,-0.088855,-0.004691,0.004691


In [41]:
df_bbands = df.ta.bbands()

In [42]:
df_bbands.columns

Index(['BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0'], dtype='object')

In [ ]:
df['BB_perL'] = (df_bbands['BBM_5_2.0']-df_bbands['BBL_5_2.0']/df_bbands['BBM_5_2.0'])

In [46]:
df['BB_perL'] = -1*((df_bbands['BBM_5_2.0']-df_bbands['BBL_5_2.0'])/df_bbands['BBM_5_2.0'])

0           NaN
1           NaN
2           NaN
3           NaN
4      0.009014
         ...   
881    0.002387
882    0.004493
883    0.004447
884    0.003801
885    0.004691
Length: 886, dtype: float64

In [47]:
df['BB_perH'] = -1*((df_bbands['BBM_5_2.0']-df_bbands['BBU_5_2.0'])/df_bbands['BBM_5_2.0'])

0           NaN
1           NaN
2           NaN
3           NaN
4     -0.009014
         ...   
881   -0.002387
882   -0.004493
883   -0.004447
884   -0.003801
885   -0.004691
Length: 886, dtype: float64

In [21]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.2.45b0
Total Indicators: 133
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_doji, cdl_inside, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, decay, decreasing, dema, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, kama, kc, kdj, kst, kurtosis, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, ssf, stdev, stoch, stochrsi, supertrend, swma, t3, tema, thermo, trend_return, trima, trix, true_range, tsi, ttm_trend, ui, uo, variance, vidya, vortex, vp, vwap, vwma, wcp, willr, wma, zlm

In [13]:
help(ta.stochrsi)

Help on function stochrsi in module pandas_ta.momentum.stochrsi:

stochrsi(close, length=None, rsi_length=None, k=None, d=None, offset=None, **kwargs)
    Stochastic (STOCHRSI)
    
    "Stochastic RSI and Dynamic Momentum Index" was created by Tushar Chande and Stanley Kroll and published in Stock & Commodities V.11:5 (189-199)
    
    It is a range-bound oscillator with two lines moving between 0 and 100.
    The first line (%K) displays the current RSI in relation to the period's
    high/low range. The second line (%D) is a Simple Moving Average of the %K line.
    The most common choices are a 14 period %K and a 3 period SMA for %D.
    
    Sources:
        https://www.tradingview.com/wiki/Stochastic_(STOCH)
    
    Calculation:
        Default Inputs:
            length=14, rsi_length=14, k=3, d=3
        RSI = Relative Strength Index
        SMA = Simple Moving Average
    
        RSI = RSI(high, low, close, rsi_length)
        LL  = lowest RSI for last rsi_length periods
  

In [22]:
help(ta.macd)

Help on function macd in module pandas_ta.momentum.macd:

macd(close, fast=None, slow=None, signal=None, offset=None, **kwargs)
    Moving Average Convergence Divergence (MACD)
    
    The MACD is a popular indicator to that is used to identify a security's trend.
    While APO and MACD are the same calculation, MACD also returns two more series
    called Signal and Histogram. The Signal is an EMA of MACD and the Histogram is
    the difference of MACD and Signal.
    
    Sources:
        https://www.tradingview.com/wiki/MACD_(Moving_Average_Convergence/Divergence)
    
    Calculation:
        Default Inputs:
            fast=12, slow=26, signal=9
        EMA = Exponential Moving Average
        MACD = EMA(close, fast) - EMA(close, slow)
        Signal = EMA(MACD, signal)
        Histogram = MACD - Signal
    
    Args:
        close (pd.Series): Series of 'close's
        fast (int): The short period. Default: 12
        slow (int): The long period. Default: 26
        signal (int

In [19]:
help(ta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, mamode=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=20, std=2
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 20
        std (int): The long period. Default: 2
        mamode (str): Two options: "sma" or "ema". Default: "ema"
        offset (int): How many 

In [35]:
df_stochrsi7 = df.ta.stochrsi(length=7, rsi_length=7)
df_stochrsi14 = df.ta.stochrsi(length=14, rsi_length=14)
df_stochrsi7['stochrsi_hist7'] = df_stochrsi7['STOCHRSIk_7_7_3_3'] - df_stochrsi7['STOCHRSId_7_7_3_3']
df_stochrsi14['stochrsi_hist14'] = df_stochrsi14['STOCHRSIk_14_14_3_3'] - df_stochrsi14['STOCHRSId_14_14_3_3']

In [37]:
df_stochrsi14.columns

Index(['STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3', 'stochrsi_hist14'], dtype='object')

In [38]:
df['stochrsi_hist7'] = (df_stochrsi7['STOCHRSIk_7_7_3_3'] - df_stochrsi7['STOCHRSId_7_7_3_3'])/100
df['stochrsi_hist14'] = (df_stochrsi14['STOCHRSIk_14_14_3_3'] - df_stochrsi14['STOCHRSId_14_14_3_3'])/100

In [39]:
df

,index,timestamp,open,high,low,close,close_per1,close_per3,ema10,ema3,rsi7,rsi14,williams7,williams14,stochrsi_hist7,stochrsi_hist14
0,1115,"01/28/2022, 01:00:00",37231.37,37517.37,36941.07,36941.07,-0.779719,-1.146761,36906.911129,37079.676534,0.515585,0.547146,-0.674335,-0.292113,NaN,NaN
1,1116,"01/28/2022, 02:00:00",36941.07,37125.71,36790.09,36794.13,-0.397769,-1.214212,36886.405469,36936.903267,0.462022,0.512311,-0.846271,-0.366593,NaN,NaN
2,1117,"01/28/2022, 03:00:00",36794.13,36889.69,36446.28,36660.46,-0.363292,-1.533411,36845.324475,36798.681634,0.416139,0.482232,-0.800035,-0.434347,NaN,NaN
3,1118,"01/28/2022, 04:00:00",36660.46,36861.48,36436.35,36508.00,-0.415870,-1.172327,36783.992752,36653.340817,0.367567,0.449795,-0.933720,-0.511625,NaN,NaN
4,1119,"01/28/2022, 05:00:00",36508.00,36692.15,36339.43,36634.00,0.345130,-0.435205,36756.721343,36643.670408,0.431542,0.480873,-0.749928,-0.447759,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,1996,"03/05/2022, 18:00:00",39466.30,39468.30,39337.11,39407.02,-0.150204,-0.211671,39369.164537,39423.083528,0.551462,0.451139,-0.586428,-0.258609,-25.331109,-2.791446
882,1997,"03/05/2022, 19:00:00",39407.02,39611.31,39361.85,39602.27,0.495470,0.584474,39411.547349,39512.676764,0.678469,0.518709,-0.027262,-0.013537,7.080938,3.161912
883,1998,"03/05/2022, 20:00:00",39602.27,39686.74,39441.29,39458.78,-0.362328,-0.019054,39420.135103,39485.728382,0.545930,0.472657,-0.576006,-0.306719,-2.523025,-2.431393
884,1999,"03/05/2022, 21:00:00",39458.78,39533.00,39383.49,39529.36,0.178870,0.310452,39439.994176,39507.544191,0.591702,0.496343,-0.397665,-0.211754,0.966424,-1.993103


In [16]:
df.ta.bbands()

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,36376.661998,36707.532,37038.402002,1.802736
...,...,...,...,...
881,39341.611085,39435.742,39529.872915,0.477389
882,39290.334799,39467.670,39645.005201,0.898635
883,39285.810604,39461.304,39636.797396,0.889445
884,39342.618991,39492.746,39642.873009,0.760276


In [17]:
df = df.append(df.ta.bbands(length=None))

In [40]:
df.ta.willr()

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
881   -25.860892
882    -1.353719
883   -30.671941
884   -21.175426
885   -43.201612
Name: WILLR_14, Length: 886, dtype: float64